<a href="https://colab.research.google.com/github/jinjukang67/AIFFEL_Project/blob/master/GD_2_Naver_review_Sentencepiece_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 영화 리뷰 감성 분석 프로젝트 : SentencePiece 적용해 보기

## Step 1. SentencePiece 설치하기
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install sentencepiece
!pip install wget

## Step 2. SentencePiece 모델 학습

### 1) 데이터 준비하기
터미널로 데이터 폴더 생성하기
```
! bash
cd /drive/MyDrive/Aiffel
mkdir -p sp_tokenizer/data
exit
```

In [ ]:
# !bash

In [ ]:
# data를 저장할 폴더 경로
data_dir = "/content/drive/MyDrive/Aiffel/sp_tokenizer/data"

In [ ]:
# 라이브러리 Import
import os
import json
import pandas as pd
import sentencepiece as spm
import wget
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import urllib.request
import csv

%matplotlib inline

In [ ]:
# filename = wget.download("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", f"{data_dir}")
# print(filename)
# filename = wget.download("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", f"{data_dir}")
# print(filename)
for f in os.listdir(data_dir):
  print(f)

### 2) Korpora가 뭔지 알아보기
> - **Korpora(한국어 코퍼스 데이터 오픈소스 파이썬 패키지)** 한번 사용해보기
> - Sentence Piece 사용하자

[Korpora 깃헙](https://github.com/ko-nlp/Korpora)<br>
[Korpora 빠른 사용법](https://ko-nlp.github.io/Korpora/ko-docs/introduction/quicktour.html)

In [ ]:
!git

In [ ]:
!pip install Korpora

In [ ]:
from Korpora import Korpora

# Korpora.fetch("all") -> Korpora가 제공하는 모든 말뭉치 내려 받을 수 있음
Korpora.fetch("nsmc") # 네이버 영화 리뷰 데이터로만 일단 단어장 사용해보자!
corpus = Korpora.load("nsmc")

In [ ]:
corpus

> - **Korpora**는 그냥 다양한 한국어 코퍼스 데이터를 쉽게 다운로드, 사용할 수만 있는 기능만 제공
> - 터미널에서 전체 데이터로 다운로드 전처리 하려면
```
korpora lmdata \
  --corpus all \
  --output_dir ~/works/lmdata
```

> - 네이버 영화 리뷰 데이터는 이미 위에서 준비 완료했으므로 1회 실습 후 더 큰 데이터를 **Korpora**에서 불러와 Vocab 만들어 실습 진행해보자

### 3) 데이터 정제하기

`rating_train_txt` document만 뽑아기
> - 데이터프레임으로 바꿔서 csv 파일로 저장한 후 
> - 다시 txt 파일 형태로 바꿔서 저장하자.

=> 오류나서 실패^^ <br>
=> 그냥 for문으로 split() 써서 가져왔는데 더 간단한 방법이 있었다.

이렇게 쉽게 가져오는 방법도 있었다^^
```
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(naver_df['document']))

```

In [ ]:
# temp = pd.read_table(path_to_file)
# temp.head()

# temp.to_csv(data_dir + '/ratings_train.csv', index=False, header=True)

In [ ]:
# csv 파일d text로 변환 후 document만 가져오기 - 실패
'''
import shutil

in_file = f"{data_dir}/ratings_train.csv"
out_file = f"{data_dir}/ratings_train_f.txt"
SEPARATOR = u"\u241D"

if not os.path.isfile(out_file):
    df = pd.read_csv(in_file, sep=SEPARATOR, engine="python")
    with open(out_file, "w") as f:
      for index, row in df.iterrows():
        f.write(row["document"]) # document만 가져오기
        f.write("\n\n\n\n") # 구분자

shutil.copy(out_file, "ratings_train_f.txt")
'''

In [ ]:
path_to_file = data_dir + "/ratings_train.txt"

with open(path_to_file, "r") as f:
    raw = f.read().splitlines()

print("Data Size:", len(raw))

print("Example:")
for sen in raw[1:100][::20]: print(">>", sen.split('	')[1])

In [ ]:
# X_train 데이터
ratings = [] 
for sen in raw[1:]: 
    ratings.append(sen.split('	')[1])

In [ ]:
# 라벨 데이터 y_train
y_train = []
for sen in raw[1:]:
    y_train.append(int(sen.split('	')[2]))

y_train[:20]

In [ ]:
len(ratings)

In [ ]:
# test 데이터셋 가져오기 1
test_path = data_dir + "/ratings_test.txt"
X_test = []
with open(test_path, "r") as f:
    raw_t = f.read().splitlines()

for sen in raw_t[1:]:
    X_test.append(sen.split('	')[1])

y_test = []
for sen in raw_t[1:]:
    y_test.append(int(sen.split('	')[2]))

print("X_test data Size:", len(X_test))
print("y_test data Size:", len(y_test))
print(X_test[:10])
print(y_test[:10])

In [ ]:
# test 데이터셋 가져오기 2
test_path = data_dir + "/ratings_test.txt"

test_data =pd.read_table(test_path)
test_data = test_data.dropna(how = 'any')
test_data.drop_duplicates(subset=['document'],inplace=True)

print(test_data.isnull().values.any()) # Null 값이 존재하는지 확인
print(test_data['document'].nunique())
print(len(test_data))

In [ ]:
# 데이터 문장 길이 분포 시각화
min_len = 999
max_len = 0
sum_len = 0

for sen in ratings:
    length = len(sen)
    if min_len > length: min_len = length
    if max_len < length: max_len = length
    sum_len += length

print("문장의 최단 길이:", min_len)
print("문장의 최장 길이:", max_len)
print("문장의 평균 길이:", sum_len // len(raw))

sentence_length = np.zeros((max_len), dtype=np.int)

print(sentence_length)

for sen in ratings:
    sentence_length[len(sen)-1] += 1 # 문장 길이 빈도수 표현

print(sentence_length)
plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

In [ ]:
# 1) 최단 길이가 0이 왜 나와?
def check_sentence_with_length(raw, length):
    count = 0

    for sen in raw:
        if len(sen) == length:
            print(sen)
            count += 1
            if count > 100: return

# check_sentence_with_length(ratings, 1)

In [ ]:
check_sentence_with_length(ratings, 2)

In [ ]:
# check_sentence_with_length(ratings, 60)

> - 최단길이가 0인걸 봐서는 공백이 들어간거 같다, 공백은 제거하자
> - 굿,굳과 같은 단어는 긍정의 의미 담고 있으니까 길이 1이라도 없애지 말자.

In [ ]:
# 문장의 길이가 같은 문장이 2000개 초과하는 sentence_length는 뭘까?
for idx, _sum in enumerate(sentence_length):
    if _sum > 3000:
        print("Outlier Index: ",idx+1)

In [ ]:
# check_sentence_with_length(ratings, 24)

In [ ]:
# 중복 있을 수도 있으니 제거해보자
min_len = 999
max_len = 0
sum_len = 0

cleaned_corpus = list(set(ratings))  # set를 사용해서 중복을 제거합니다.
print("Data Size:", len(cleaned_corpus))

for sen in cleaned_corpus:
    length = len(sen)
    if min_len > length: min_len = length
    if max_len < length: max_len = length
    sum_len += length

print("문장의 최단 길이:", min_len)
print("문장의 최장 길이:", max_len)
print("문장의 평균 길이:", sum_len // len(cleaned_corpus))

sentence_length = np.zeros((max_len), dtype=np.int)

for sen in cleaned_corpus:   # 중복이 제거된 코퍼스 기준
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

In [ ]:
print("중복된 값: ",len(ratings) - len(cleaned_corpus))

> - 150,000 개에서 중복된 데이터 3817개를 제거하여 146,183개로 줄었다.

1. 네이버 영화 후기의 경우 길이가 1,2 로 짧아도 감정이 드러나기 때문에 min_len 은 1로 하자.
2. 후기니까 문장 길이 80이상은 제거하자.

In [ ]:
max_len = 80
min_len = 1

# 길이 조건에 맞는 문장만 선택합니다.
filtered_corpus = [s for s in cleaned_corpus if (len(s) < max_len) & (len(s) >= min_len)]

# 분포도를 다시 그려봅니다. -> 최종적인 데이터 분포
sentence_length = np.zeros((max_len), dtype=np.int)

for sen in filtered_corpus:
    sentence_length[len(sen)-1] += 1

plt.bar(range(max_len), sentence_length, width=1.0)
plt.title("Sentence Length Distribution")
plt.show()

### Konlpy로 형태소 분석해보면?

In [ ]:
# mecab 설치하기
!apt-get update 
!apt-get install g++ openjdk-8-jdk 
!pip install konlpy JPype1-py3 
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
#  mecab.morphs()를 사용해서 형태소 분석
from konlpy.tag import Mecab

mecab = Mecab()

def mecab_split(sentence):
    return mecab.morphs(sentence)

mecab_corpus = []

for kor in filtered_corpus:
    mecab_corpus.append(mecab_split(kor))

In [ ]:
# corpus: Tokenized Sentence's List
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [ ]:
# 단어 사전 길이 확인
mecab_tensor, mecab_tokenizer = tokenize(mecab_corpus)

print("MeCab Vocab Size:", len(mecab_tokenizer.index_word))

mecab 테스트 해보기

In [ ]:
# Case 1
texts = mecab_tokenizer.sequences_to_texts([mecab_tensor[300]])

print(texts[0])

In [ ]:
# Case 2
sentence = ""

for w in mecab_tensor[300]:
    if w == 0: continue
    sentence += mecab_tokenizer.index_word[w] + ""

print(sentence)

In [ ]:
test = mecab_tokenizer.texts_to_sequences("줄거리에 다써놨어 ㅋㅋㅋ ㅁ ㅣ친 ㄴ ㅔ이버")
test

In [ ]:
result = mecab_tokenizer.sequences_to_texts(test)
print(result)

# 불용어 제거 어디서 어떻게 하지?

### 4) Sentence Piece를 이용해서 Vocab 파일을 만들기


In [ ]:
data_dir

In [ ]:
filtered_corpus[:10]

In [ ]:
import sentencepiece as spm
import os
temp_file = data_dir + '/ratings_train_temp4.txt'

vocab_size = 10000
model_name = 'rating_cor4'

with open(temp_file, 'w') as f:
    for row in filtered_corpus:   # 이전 스텝에서 정제했던 corpus를 활용합니다.
        f.write(str(row) + '\n')

spm.SentencePieceTrainer.Train(
    "--input={} --model_prefix={} --vocab_size={}".format(temp_file, model_name, vocab_size) +
    " --model_type=char" + # Train에서  --model_type = 'unigram'이 디폴트, --model_type = 'bpe' 로 옵션을 주어 변경 가능.
    " --max_sentence_length=80" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" ) # end of sequence (3)

!ls -l rating_cor4*

In [ ]:
s = spm.SentencePieceProcessor()
s.Load('rating_cor4.model')

# SentencePiece를 활용한 sentence -> encoding
tokensIDs = s.EncodeAsIds('줄거리에 다써놨어 ㅋㅋㅋ ㅁ ㅣ친 ㄴ ㅔ이버')
print(tokensIDs)

# SentencePiece를 활용한 sentence -> encoded pieces
print(s.SampleEncodeAsPieces('줄거리에 다써놨어 ㅋㅋㅋ ㅁ ㅣ친 ㄴ ㅔ이버',1, 0.0))

# SentencePiece를 활용한 encoding -> sentence 복원
print(s.DecodeIds(tokensIDs))

## Step 3. Tokenizer 함수 작정
-  위에서 훈련시킨 SentencePiece를 활용
> 1. 매개변수로 토큰화된 문장의 list를 전달하는 대신 **온전한 문장**의 list를 전달한다.
> 2. **생성된 vocab 파일** 읽어와 `{ <word> : <idx> }` 형태를 가지는 `word_index` 사전과 `{ <idx> : <word>}` 형태를 가지는 `index_word` 사전을 생성하고 함께 **반환**합니다.
> 3. 리턴값인 `tensor` 는 앞의 함수와 동일하게 토큰화한 후 Encoding된 문장입니다. 바로 학습에 사용할 수 있게 Padding은 당연히 해야합니다.

In [ ]:
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./rating_cor4.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='pre') # RNN에는 'pre'가 더 유리하다

    return tensor, word_index, index_word

In [ ]:
#sp_tokenize(s, corpus) 사용예제

my_corpus = ['나는 밥을 먹었습니다.', '그러나 여전히 ㅠㅠ 배가 고픕니다...']
tensor, word_index, index_word = sp_tokenize(s, my_corpus)
print("Tensor:",tensor)
print("Word_index:",word_index)
print("Index_word:",index_word)

## 데이터 로더 함수 만들기
- 데이터 정규표현식 전처리 함수 만들기
- 데이터 중복 제거
- 데이터 결측치 제거
- 불용어 처리
- sp_tokenize로 



> - 한글 불용어 파이썬 라이브러리 없기때문에 직접 불용어 사전 만들어 제거하자!
> -  한국어 불용어 리스트 100개 txt 파일 쓰는 방법도 있음!

[한국어 불용어 리스트 100개](https://bab2min.tistory.com/544)

In [ ]:
# raw 데이터 불러와서 시작하자
path_to_file = data_dir + "/ratings_train.txt"
test_path = data_dir + "/ratings_test.txt"

train_d = pd.read_table(path_to_file)
test_d = pd.read_table(test_path)

train_d.head()

In [ ]:
import re

# 불용어 사전 만들기
stopwords = set(['하','아','의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'])

def preprocess_data(sentence):
    # 한글 및 공백 이외 다른 문자 모두 제거.
    sentence = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sentence)
    return sentence

def load_data(train_data, test_data, num_words=10000):

    # train/test의 중복, 결측치 제거, 한글 및 공백 이외 제거
    train_data.drop_duplicates(subset=['document'], inplace=True) 
    train_data['document'] = train_data['document'].apply(lambda x: preprocess_data(str(x)))
    train_data = train_data.dropna(how = 'any')

    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data['document'] = test_data['document'].apply(lambda x: preprocess_data(str(x)))
    test_data = test_data.dropna(how = 'any') 

    train_list = []
    for sentence in train_data['document']:
        # temp_X = s.encode_as_pieces(sentence) # 토큰화
        # temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        # temp_X = s.encode_as_ids(sentence)
        train_list.append(sentence)

    test_list = []
    for sentence in test_data['document']:
        # temp_X = s.encode_as_pieces(sentence) # 토큰화
        # temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
        # temp_X = s.encode_as_ids(sentence)
        test_list.append(sentence)
    
    X_train, word_to_index, index_to_word = sp_tokenize(s, train_list)
    X_test , _,_ = sp_tokenize(s, test_list)

        
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index
    
X_train, y_train, X_test, y_test, word_to_index = load_data(train_d, test_d) 

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
# word_to_index

## Step 4. 모델 구성하기
- 1. LSTM 
- 2. 1-D CNN
- 3. GlobalMaxPooling1D() 레이어 하나만 쓰기
- 4. Transformer 이용 (시간 더 있을 시)

**LSTM 모델 구성**

In [ ]:
# LSTM 모델
vocab_size = 8000
word_vector_dim = 32 

model_lstm = tf.keras.Sequential()
model_lstm.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_lstm.add(tf.keras.layers.LSTM(64))
model_lstm.add(tf.keras.layers.Dense(32, activation='relu'))
model_lstm.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_lstm.summary()

**1-D CNN 모델 구성**

In [ ]:
# 1-D CNN으로 만든 모델 구현

model_1dc = tf.keras.Sequential()
model_1dc.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_1dc.add(tf.keras.layers.Conv1D(32, 7, activation='relu')) # word_vector_dim과 맞춰봤습니다.
model_1dc.add(tf.keras.layers.MaxPooling1D(5))
model_1dc.add(tf.keras.layers.Conv1D(32, 7, activation='relu')) # word_vector_dim과 맞춰봤습니다.
model_1dc.add(tf.keras.layers.GlobalMaxPooling1D())
model_1dc.add(tf.keras.layers.Dense(16, activation='relu'))
model_1dc.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_1dc.summary()

**GlobalMaxPooling1D() 레이어만 사용한 모델**





In [ ]:
# GlobalMaxPolling만 있는 것 

model_gmp = tf.keras.Sequential()
model_gmp.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_gmp.add(tf.keras.layers.GlobalMaxPooling1D())
model_gmp.add(tf.keras.layers.Dense(16, activation='relu'))
model_gmp.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_gmp.summary()

**Validations set 구분**

In [ ]:
# train_test_split으로 나눠보겠습니다.
from sklearn.model_selection import train_test_split

xx_train, xx_validation, yy_train, yy_validation = train_test_split(X_train,
                                                                   y_train,
                                                                   test_size=0.2,
                                                                   random_state=2022)

In [ ]:
# 에포크
EPOCHS = 10

### 1. LSTM


In [ ]:
model_lstm.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model_lstm.fit(xx_train,
                         yy_train,
                         epochs=EPOCHS,
                         batch_size=512,
                        validation_data=(xx_validation, yy_validation),
                        verbose=2)

In [ ]:
# test 셋으로 평가해보기
results = model_lstm.evaluate(X_test, y_test, verbose=2)

print(results)

In [ ]:
# 그래프로 그려보기

acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

plt.figure(figsize=(12,8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='training')
plt.plot(val_acc, 'o', label='validation')
plt.xlabel('epocks')
plt.ylabel('accuracy', fontsize=20)
plt.legend(fontsize=20)

plt.subplot(1, 2, 2)
plt.plot(loss, label='training')
plt.plot(val_loss, 'o', label='validation')
plt.xlabel('epocks')
plt.ylabel('loss', fontsize=20)
plt.legend(fontsize=20)

plt.show()

### 2. 1-D CNN

In [ ]:
model_1dc.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model_1dc.fit(xx_train,
                         yy_train,
                         epochs=EPOCHS,
                         batch_size=512,
                        validation_data=(xx_validation, yy_validation),
                        verbose=2)

In [ ]:
# test 셋으로 평가해보기
results = model_1dc.evaluate(X_test, y_test, verbose=2)

print(results)

In [ ]:
# 그래프로 그려보기

acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

plt.figure(figsize=(12,8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='training')
plt.plot(val_acc, 'o', label='validation')
plt.xlabel('epocks')
plt.ylabel('accuracy', fontsize=20)
plt.legend(fontsize=20)

plt.subplot(1, 2, 2)
plt.plot(loss, label='training')
plt.plot(val_loss, 'o', label='validation')
plt.xlabel('epocks')
plt.ylabel('loss', fontsize=20)
plt.legend(fontsize=20)

plt.show()


### 3. Global Max Pooling

In [ ]:
model_gmp.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model_gmp.fit(xx_train,
                         yy_train,
                         epochs=EPOCHS,
                         batch_size=512,
                        validation_data=(xx_validation, yy_validation),
                        verbose=2)

In [ ]:
# test 셋으로 평가해보기
results = model_gmp.evaluate(X_test, y_test, verbose=2)

print(results)

In [ ]:
# 그래프로 그려보기

acc = history.history['accuracy']
loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

plt.figure(figsize=(12,8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='training')
plt.plot(val_acc, 'o', label='validation')
plt.xlabel('epocks')
plt.ylabel('accuracy', fontsize=20)
plt.legend(fontsize=20)

plt.subplot(1, 2, 2)
plt.plot(loss, label='training')
plt.plot(val_loss, 'o', label='validation')
plt.xlabel('epocks')
plt.ylabel('loss', fontsize=20)
plt.legend(fontsize=20)

plt.show()

# 실험 결과
1. vocab size = 8000, model_type = bpe 일 때

> LSTM 
```
1537/1537 - 6s - loss: 0.4196 - accuracy: 0.8348 - 6s/epoch - 4ms/step
[0.41956353187561035, 0.8348386883735657]
```

> 1-D CNN
```
1537/1537 - 4s - loss: 1.1662 - accuracy: 0.7817 - 4s/epoch - 3ms/step
[1.1662278175354004, 0.7817038893699646]
```

> Global Max Pooling
```
1537/1537 - 3s - loss: 0.4817 - accuracy: 0.8334 - 3s/epoch - 2ms/step
[0.4816548824310303, 0.8333536982536316]
```

2. vocab size = 10000, model_type = bpe 일 때

> LSTM 
```
1537/1537 - 7s - loss: 0.4194 - accuracy: 0.8371 - 7s/epoch - 5ms/step
[0.4194413721561432, 0.8370763659477234]
```

> 1-D CNN
```
1537/1537 - 5s - loss: 1.1183 - accuracy: 0.7455 - 5s/epoch - 3ms/step
[1.1182972192764282, 0.745514452457428]
```

> Global Max Pooling
```
1537/1537 - 4s - loss: 0.4168 - accuracy: 0.8358 - 4s/epoch - 3ms/step
[0.4167833626270294, 0.8357744216918945]
```



3. vocab size = 10000, model_type = unigram(default) 일 때

> LSTM 
```
1537/1537 - 7s - loss: 0.3842 - accuracy: 0.8381 - 7s/epoch - 5ms/step
[0.3842264413833618, 0.8380731344223022]
```

> 1-D CNN
```
1537/1537 - 5s - loss: 1.1931 - accuracy: 0.7818 - 5s/epoch - 3ms/step
[1.1930625438690186, 0.781846284866333]
```

> Global Max Pooling
```
1537/1537 - 4s - loss: 0.4721 - accuracy: 0.8319 - 4s/epoch - 3ms/step
[0.4721464216709137, 0.8319296836853027]
```



4. vocab size = 10000, model_type = char 일 때

> LSTM 
```
1537/1537 - 7s - loss: 0.3872 - accuracy: 0.8224 - 7s/epoch - 4ms/step
[0.3871708810329437, 0.8224093914031982]
```

> 1-D CNN -> 오버피팅 발생!!!
```
1537/1537 - 4s - loss: 0.4319 - accuracy: 0.8254 - 4s/epoch - 3ms/step
[0.43188703060150146, 0.8253793716430664]
```

> Global Max Pooling
```
1537/1537 - 3s - loss: 0.4471 - accuracy: 0.7967 - 3s/epoch - 2ms/step
[0.4470667541027069, 0.7966556549072266]
```

**=> 형태소 분석기를 썼을 때와 정확도 차이가 크게 안난다.**

# 회고
---
> - 처음에 데이터 불러오는 방법이 여러가지고 코드가 익숙하지 않아 많이 헤맸다. 이제 이런 일 없도록 해야겠다.
> - NLP 텍스트 전처리 과정이 명확하게 머리에 안그려지는 상황에서 SentenciePiece를 활용하려고 하다보니 데이터 로더 함수를 짜는데도 시간이 좀 소요되었다. 텍스트 전처리할때 토크나이저 패키지를 잘 활용할 줄 아는 것도 효율적으로 시간을 쓸 수 있는 방법이라는 것을 깨달았다.
> - Korpora 라는 오픈소스 패키지를 발겼했다. 한국어 텍스트 데이터를 바로 불러와서 쓸 수 있는 유용한 패키지 같은데 다음에 한번 활용해봐야겠다.
> - **근데 아직 SentencePiece로 만든 토크나이저 함수를 활용할 때 불용어 제거를 어느 부분에서 해야할 지 모르겠다. 이부분은 자연어처리 고수분에게 물어보고 넘어가자.**
> - 불용어 제거할 때, 한국어는 불용어 제거 패키지가 없어 직접 불용어 사전을 만들어야 한다고 했다. 하지만 한국어 불용어 모음 텍스트 파일을 공유하기도 하니 불용어 사전을 만들 때 이것을 가져와 쓰거나 아니면 필요한 부분만 가져와서 사용하는 것도 좋을 것 같다.
> - 단어장 만들 때 vocab_size 와 model_type을 바꿔가면서 정확도가 어떻게 달라지는 지 실험을 했는데 LSTM & Global Max Pooling 과 1-D CNN이 반비례 처럼 달라졌다. 같은 모델 타입(bpe)에서 vocab_size를 더 크게 잡았을 땐, LSTM 과 GMP는 정확도가 조금 증가하였지만 1-D CNN은 오히려 더 떨어졌다. 그리고 마지막에 model_type default인 unigram으로 단어장을 만들어 학습시켰을 때는 LSTM, GMP는 정확도가 떨어졌지만 1-D CNN은 정확도가 80%가 넘으면 많이 올랐다.
> - 단순한 모델을 사용했기 때문에 다음엔 더 공부해서 Transformer와 BERT도 사용하고 워드 벡터 차원, 에포크 수도 늘려서 학습시켜볼 예정이다.
> - 그때는 오버피팅도 해결하면서 학습시켜보는걸로!

> **다음에 할 것을 정리해보자면**
> 1. 단어장을 네이버 영화 리뷰 말고 더 큰 한국어 텍스트로 만들어보기
> 2. 불용어 제거하는 방법 찾아서 불용어 제거하고 Sentencepiece 토크나이저 돌리기
> 3. 좀 더 복잡한 모델 transformer로 돌리기
> 4. 오버피팅나면 정규화해서 학습시키기